In [1]:
import torch
import numpy as np
import pandas as pd


In [2]:
preds_path = "/mnt/code/users/xuziyang/PromptBias/outputs/openprompt/manual_prompt/bert-base-cased/LAMA/debias_answer_type_tokens/origin_embedding/preds.pt"
preds = torch.load(preds_path)
relation_csv = "/mnt/code/users/xuziyang/PromptBias/code/analysis/relation_type.csv"
relations_meta = pd.read_csv(relation_csv)

In [5]:
# 对于每个relation都寻找一个rectified examples
selected_num = 5
selected_rows = []
for relation in preds["WIKI-UNI"].keys():
    df = pd.DataFrame(preds["WIKI-UNI"][relation]["data"])
    # 定位preds中数量最多的类别
    # 计算每个元素的频数
    value_counts = df['raw_preds'].value_counts()
    # 找出频数最高的元素
    bias_label = value_counts.idxmax()
    # 找到debias矫正成功的examples
    # 根据条件筛选行
    filtered_rows = df[(df['debiased_preds'] == df['obj_labels']) & (df['raw_preds'] == bias_label)]
    if not filtered_rows.empty and len(filtered_rows)>=selected_num:
        selected_row = filtered_rows.iloc[:selected_num].copy()
        # print(selected_row, type(selected_row))
        template = relations_meta[relations_meta["relation"] == relation ]["tempalte"].item()
        # print(template, type(template))
        selected_row['query'] = [template.replace("[X]", selected_row["sub_label"].values[i]).replace("[Y]","[MASK]") for i in range(selected_num)]
        selected_row = selected_row
        selected_rows.append(selected_row)
        # print(selected_row)   

# 合并选取的行成为一个新的 DataFrame
merged_df = pd.concat(selected_rows)



In [6]:
selected_row
merged_df[:20]

,index,sub_label,obj_labels,raw_preds,debiased_preds,query
7,7,Union of South Africa,Dutch,English,Dutch,The official language of Union of South Africa...
8,8,Suriname,Dutch,English,Dutch,The official language of Suriname is [MASK] .
35,35,Guinea-Bissau,Portuguese,English,Portuguese,The official language of Guinea-Bissau is [MAS...
36,36,Southern Rhodesia,English,English,English,The official language of Southern Rhodesia is ...
47,47,Panama,Spanish,English,Spanish,The official language of Panama is [MASK] .
5,5,Don Minnick,pitcher,center,pitcher,Don Minnick plays in [MASK] position .
20,20,Jack McAdams,pitcher,center,pitcher,Jack McAdams plays in [MASK] position .
33,33,Jerry Reuss,pitcher,center,pitcher,Jerry Reuss plays in [MASK] position .
65,65,Harry Jordan,pitcher,center,pitcher,Harry Jordan plays in [MASK] position .
70,70,Tim Redding,pitcher,center,pitcher,Tim Redding plays in [MASK] position .


In [79]:
merged_df

,697,1970,2076,20,489,1061,7,8,35,5,...,26,231,245,264,360,362,519,41,84,348
index,697.0,1970.0,2076.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub_label,Broda Otto Barnes,Mark Prisk,Emily Faithfull,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
obj_labels,Missouri,Cornwall,Surrey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
raw_preds,London,London,London,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
debiased_preds,Missouri,Cornwall,Surrey,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub_label,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,April 1812,Lyons (Rosevears),sulcus fragmentosus
obj_labels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1812,Lyons,brain
raw_preds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,France,France
debiased_preds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1812,Lyons,brain


In [59]:
df.keys()

Index(['index', 'sub_label', 'obj_labels', 'raw_preds', 'debiased_preds'], dtype='object')

In [66]:
merged_df

,sub_label,obj_labels,raw_preds,debiased_preds,query
0,Broda Otto Barnes,Missouri,London,Missouri,Broda Otto Barnes was born in [MASK] .
1,Victor Hall,England,London,England,Victor Hall died in [MASK] .
2,Union of South Africa,Dutch,English,Dutch,The official language of Union of South Africa...
3,Don Minnick,pitcher,center,pitcher,Don Minnick plays in [MASK] position .
4,I Married a Princess,Lifetime,ABC,Lifetime,I Married a Princess was originally aired on [...
5,province of Brittany,Normandy,France,Normandy,province of Brittany shares border with [MASK] .
6,Elektro,electricity,Alexander,electricity,Elektro is named after [MASK] .
7,Blind Justice,English,English,English,The original language of Blind Justice is [MAS...
8,South African Football Association,FIFA,FIFA,FIFA,South African Football Association is a member...
9,Charlie Sifford,golf,psychology,golf,Charlie Sifford works in the field of [MASK] .


In [28]:
relations_meta

,relation,type,tempalte,label
0,P1001,N-M,[X] is a legal term in [Y] .,applies to jurisdiction
1,P101,N-M,[X] works in the field of [Y] .,field of work
2,P102,N-1,[X] is a member of the [Y] political party .,member of political party
3,P103,N-1,The native language of [X] is [Y] .,native language
4,P106,N-M,[X] is a [Y] by profession .,occupation
5,P108,N-M,[X] works for [Y] .,employer
6,P127,N-1,[X] is owned by [Y] .,owned by
7,P1303,N-M,[X] plays [Y] .,instrument
8,P131,N-1,[X] is located in [Y] .,located in the administrative territorial entity
9,P136,N-1,[X] plays [Y] music .,genre


In [14]:
df.keys()

Index(['index', 'sub_label', 'obj_labels', 'raw_preds', 'debiased_preds'], dtype='object')

In [12]:
len(preds["WIKI-UNI"].keys())

41

In [8]:
preds.keys()

dict_keys(['LAMA', 'WIKI-UNI', 'LAMA-WHU'])

,relation,type,tempalte,label
0,P1001,N-M,[X] is a legal term in [Y] .,applies to jurisdiction
1,P101,N-M,[X] works in the field of [Y] .,field of work
2,P102,N-1,[X] is a member of the [Y] political party .,member of political party
3,P103,N-1,The native language of [X] is [Y] .,native language
4,P106,N-M,[X] is a [Y] by profession .,occupation
5,P108,N-M,[X] works for [Y] .,employer
6,P127,N-1,[X] is owned by [Y] .,owned by
7,P1303,N-M,[X] plays [Y] .,instrument
8,P131,N-1,[X] is located in [Y] .,located in the administrative territorial entity
9,P136,N-1,[X] plays [Y] music .,genre
